In [1]:
import numpy as np
import os, json
from argparse import Namespace
from flat_reach_RL import *
from nnet_util import torch_model_to_nnet, onnx_to_nnet
import onnxruntime

pybullet build time: Oct  8 2020 00:11:27


In [2]:
demo_path = "/home/wth/dev/Composable_Agent_Toolbox/examples/output/FlatReach_near_C200_H32_Nm100.0_lr0.001"
demo_ep = 300

with open(os.path.join(demo_path, "args.json"), "r") as infile:
    config = json.load(infile)
args_saved = Namespace(**config)

args_saved.demo = True
args_saved.demo_path = demo_path
args_saved.demo_ep = demo_ep

args = args_saved

STATE_DIMENSION = 8
ACTION_SPACE_SIZE = 17

device = 'cuda'

In [3]:
Q = QNetwork(env_name=args.env_name,
            state_dim=STATE_DIMENSION,
            num_of_actions=ACTION_SPACE_SIZE,
            hidden_size=args.hidden_size).to(device)

checkpt_path = os.path.join(args.demo_path, 'checkpts', '{}_{}.pt'.format(args.env_name, args.demo_ep))
Q.load_model_weights(checkpt_path)
Q.eval()

QNetwork(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=17, bias=True)
  )
)

In [4]:
dummy_input = torch.randn(1, STATE_DIMENSION).to(device)
input_names = ["dummy_input"]
output_names = ["output"]

torch.onnx.export(Q, 
                  dummy_input,
                  f"{demo_ep}.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  export_params=True,
                  )

In [9]:
torch_model_to_nnet(Q, f"{demo_ep}.nnet", decimal=8) # saving 8 digits. More can be used, but that requires more more space.

Converted pytorch model to an NNet model at 300.nnet


In [10]:
onnx_to_nnet(f"{demo_ep}.onnx", f"{demo_ep}.nnet", decimal=8)

Converted ONNX model at 300.onnx
    to an NNet model at 300.nnet


In [11]:
def onnx_inference(file, input):
    sess = onnxruntime.InferenceSession(
        file, providers=onnxruntime.get_available_providers())
    input_name = sess.get_inputs()[0].name
    pred_onx = sess.run(None, {input_name: input.astype(np.float32)})[0]
    print(pred_onx)

input = np.array([[1,2,3,4,5,6,7,8]])
onnx_inference("300.onnx", input)

[[930.37695 932.53015 937.174   936.2096  940.66046 927.5235  930.0774
  928.9899  928.7441  941.59045 945.49884 953.39    935.05896 929.7237
  925.0195  935.30005 934.61084]]


In [ ]:
# Test nnet in julia using the following code:
# the output could be slightly different due to digits used for saving nnet

# using NeuralVerification
# net = read_nnet("300.nnet")
# input = [1,2,3,4,5,6,7,8]
# compute_output(net, input)